In [ ]:
import re

import pandas as pd
from pathlib import Path
from ploting import *

In [ ]:
root_path = "perlmutter"

In [ ]:
name_re = re.compile("(weak|strong)_scaling-d_(\d+)-n_(\d+)-([\w_]+)_comm-(gpu|cpu)-nodes_(\d+)")

In [ ]:
time_re = re.compile("DURATION:\s*(\d+(\.\d+)?(e[+-]\d+)?)")

In [ ]:
data = []
for file in Path(f"{root_path}/data").iterdir():
    if m := name_re.search(file.name):
        with open(file, "r") as input:
            if t_m := time_re.search(input.read()):
                duration = t_m.group(1)
                data.append({"scaling": m.group(1), "dim": m.group(2), "n": m.group(3), "comm_pattern": m.group(4),
                             "type": m.group(5), "nodes": m.group(6), "time": duration})
df = pd.DataFrame(data).drop(columns=["scaling", "type"]).astype({"dim": int, "n": int, "nodes": int, "time": float})
df

In [ ]:
piv = df.pivot(index="nodes", columns=["dim", "comm_pattern", "n"], values="time")
piv = piv[piv.columns.sort_values()]
piv

In [ ]:
for dim in piv.columns.levels[0]:
    _df = piv[dim]
    for gc in piv.columns.levels[1]:
        fig, ax = plt.subplots(dpi=300)
        ax.set_prop_cycle(default_cycler if gc == "stencil" else dashed_cylcer)
        ax.set_title(f"{dim}D")
        _df[gc].apply(lambda s: s[1] / s).plot(ax=ax, legend=False, logx=True)
        ax.legend([f"{n:g}" for n in piv.columns.levels[2]])
    #fig.savefig(f"{root_path}/img/{dim}d.png")